# Stage 1 - Make and train the model

In [1]:
import pandas as pd
import numpy as np
import torch

## dataset generation
# lets say we want to train a function that returns max(x,y)

input_data = pd.DataFrame({
    'x': np.random.uniform(-1000, 1000, 10000),
    'y': np.random.uniform(-1000, 1000, 10000)
})
output_data = pd.DataFrame({
    'output': np.maximum(input_data['x'], input_data['y'])
})


df = pd.concat([input_data, output_data], axis=1)

df.describe()

,x,y,output
count,10000.000000,10000.000000,10000.000000
mean,2.870207,1.393313,339.856861
std,578.634541,582.206049,472.369702
min,-999.638201,-999.427704,-983.123838
25%,-496.774421,-501.091755,9.998594
50%,4.168013,2.809366,425.599138
75%,501.408226,517.594201,740.628323
max,999.711480,999.982514,999.982514


In [2]:
# model definition and training

class SimpleModel(torch.nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.linear1 = torch.nn.Linear(2, 10)
        self.linear2 = torch.nn.Linear(10, 1)

    def forward(self, x):
        x = torch.relu(self.linear1(x))
        x = self.linear2(x)
        return x
    
    # this method is necessary for the model to be evaluated by poly-lithic

    def evaluate(self, x):
        # x will be a dicrt of keys and values
        # {"x": x, "y": y}
        input_tensor = torch.tensor([x['x'], x['y']], dtype=torch.float32)
        # you may want to do somethinf more complex here
        output_tensor = self.forward(input_tensor)
        # return a dictionary of keys and values
        return {"output": output_tensor.item()}        
    
# model training
model = SimpleModel()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.MSELoss()
batch_size = 32

# convert data to torch tensors
X = torch.tensor(df[['x', 'y']].values, dtype=torch.float32)
y = torch.tensor(df[['output']].values, dtype=torch.float32)
# training loop
for epoch in range(100):
    for i in range(0, len(X), batch_size):
        X_batch = X[i:i+batch_size]
        y_batch = y[i:i+batch_size]

        # forward pass
        y_pred = model(X_batch)

        # compute loss
        loss = loss_fn(y_pred, y_batch)

        # backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch}, Loss: {loss.item()}')
    
# save the model
torch.save(model.state_dict(), 'local/model.pth')
    

Epoch 0, Loss: 4.1773576736450195
Epoch 1, Loss: 0.21251170337200165
Epoch 2, Loss: 0.2396649420261383
Epoch 3, Loss: 0.14902222156524658
Epoch 4, Loss: 0.11189847439527512
Epoch 5, Loss: 0.08143521100282669
Epoch 6, Loss: 0.07565190643072128
Epoch 7, Loss: 0.06692548841238022
Epoch 8, Loss: 0.043258484452962875
Epoch 9, Loss: 0.036197226494550705
Epoch 10, Loss: 0.054438211023807526
Epoch 11, Loss: 0.02050456404685974
Epoch 12, Loss: 0.013617522083222866
Epoch 13, Loss: 0.020160654559731483
Epoch 14, Loss: 0.12236356735229492
Epoch 15, Loss: 0.006516922730952501
Epoch 16, Loss: 8.002128601074219
Epoch 17, Loss: 1.2831875085830688
Epoch 18, Loss: 3.0018365383148193
Epoch 19, Loss: 0.2687535285949707
Epoch 20, Loss: 4.842458724975586
Epoch 21, Loss: 0.012875671498477459
Epoch 22, Loss: 0.06681492924690247
Epoch 23, Loss: 10.468391418457031
Epoch 24, Loss: 0.3817182779312134
Epoch 25, Loss: 0.20693449676036835
Epoch 26, Loss: 0.38488632440567017
Epoch 27, Loss: 0.03862190246582031
Epoch 

In [3]:
# test the model
test_data = pd.DataFrame({
    'x': np.random.uniform(-1000, 1000, 100),
    'y': np.random.uniform(-1000, 1000, 100)
})
test_X = torch.tensor(test_data[['x', 'y']].values, dtype=torch.float32)
test_y = torch.tensor(np.maximum(test_data['x'], test_data['y']), dtype=torch.float32)
test_y_pred = model(test_X)
test_loss = loss_fn(test_y_pred, test_y)
print(f'Test Loss: {test_loss.item()/len(test_X)}')

# print sample
for i in range(10):
    print(f"Input: {test_data.iloc[i].values}, Predicted: {test_y_pred[i].item()}, Actual: {test_y[i].item()}")

Test Loss: 4254.9275
Input: [ 98.19370686 545.76953676], Predicted: 545.989990234375, Actual: 545.76953125
Input: [-33.56557292  78.23007306], Predicted: 78.2187271118164, Actual: 78.23007202148438
Input: [445.50908726 738.20504473], Predicted: 738.5706176757812, Actual: 738.2050170898438
Input: [-224.36883391 -855.20430802], Predicted: -223.91690063476562, Actual: -224.36883544921875
Input: [  -7.1227906  -948.68151075], Predicted: -7.091699600219727, Actual: -7.122790813446045
Input: [ 867.89452037 -960.86086584], Predicted: 865.7548217773438, Actual: 867.89453125
Input: [  -1.53737919 -874.7682752 ], Predicted: -1.517396330833435, Actual: -1.5373791456222534
Input: [-789.66240356  864.50937532], Predicted: 864.3361206054688, Actual: 864.5093994140625
Input: [878.03037571 788.22255001], Predicted: 876.9638671875, Actual: 878.0303955078125
Input: [-272.58051688  796.14193243], Predicted: 796.4116821289062, Actual: 796.1419067382812


/home/gbm96348/nfs_home/ml_env/lib/python3.11/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


# Stage 2a - Write a poly-lithic compatible model  - from local file system

Look inside `examples/base/local/model_definition.py`

The file is composed of a model factory and a model definition. Which we point to in the deployment.yaml file.


Model definition as beofre:

``` python
class SimpleModel(torch.nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.linear1 = torch.nn.Linear(2, 10)
        self.linear2 = torch.nn.Linear(10, 1)

    def forward(self, x):
        x = torch.relu(self.linear1(x))
        x = self.linear2(x)
        return x
    
    # this method is necessary for the model to be evaluated by poly-lithic
    
    def evaluate(self, x: dict) -> dict:
        # x will be a dicrt of keys and values
        # {"x": x, "y": y}
        input_tensor = torch.tensor([x['x'], x['y']], dtype=torch.float32)
        # you may want to do somethinf more complex here
        output_tensor = self.forward(input_tensor)
        # return a dictionary of keys and values
        return {"output": output_tensor.item()}
```

Model factory:

``` python
class ModelFactory:
    
    # can do more complex things here but we will just load the model from a locally saved file
    def __init__(self):
        self.model = SimpleModel()
        self.model = self.model.load_state_dict(torch.load('model.pth'))
        # other magic init stuff here
        print("ModelFactory initialized")
    
    # this method is necessary for the model to be retrieved by poly-lithic
    def get_model(self):
        return self.model
```



In [4]:
# lets check our import works
from local.model_definition import ModelFactory

factory = ModelFactory()

model = factory.get_model()


sample_input = {
    "x": -20,
    "y": 1000.0
}
sample_output = model.evaluate(sample_input)
print(f"Input: {sample_input}, Output: {sample_output}")


PYTHONPATH set to: /home/gbm96348/nfs_home/lume-deployment
ModelFactory initialized
Input: {'x': -20, 'y': 1000.0}, Output: {'output': -169.1890869140625}


In this case our `deployment_config.yaml` file will look like this:

``` yaml
deployment:
  type: "continuous"
  rate: 1 #seconds

modules: 
  p4p_server:
    name: "p4p_server"
    type: "interface.p4p_server"
    pub: 
      - "in_interface"
    sub: 
      - "get_all"
      - "out_transformer" # look we can do this now!               
    module_args: None # defines what arguments to pass to the module observer, if any this can inform unpacking etc
    config: 
      EPICS_PVA_NAME_SERVERS: "localhost:5075"
      variables:
        ML:LOCAL:TEST_B:
          proto: pva
          name: ML:LOCAL:TEST_B 
        ML:LOCAL:TEST_A:
          proto: pva
          name: ML:LOCAL:TEST_A
        ML:LOCAL:TEST_S:
          proto: pva
          name: ML:LOCAL:TEST_S
  
  input_transformer:
    name: "input_transformer"
    type: "transformer.SimpleTransformer"
    pub: "in_transformer"
    sub: "in_interface"
    module_args: None
    config:
      symbols:
        - "ML:LOCAL:TEST_B"
        - "ML:LOCAL:TEST_A"
      variables:
        x: # note this is where we change the name to match what the model expects
          formula: "ML:LOCAL:TEST_A"
        y: 
          formula: "ML:LOCAL:TEST_B"
  model:
    name: "model"
    type: "model.SimpleModel"
    pub: "model"
    sub: "in_transformer"
    module_args: None
    config:
      type: "LocalModelGetter"
      args: 
        model_path: "examples/base/local/model_definition.py"           # path to the model definition
        model_factory_class: "ModelFactory"                             # class that you use to create the model
      variables:
        max:
          type: "scalar"
  
  output_transformer:
    name: "output_transformer"
    type: "transformer.SimpleTransformer"
    pub: "out_transformer"
    sub: "model"
    module_args: 
      unpack_data: True
    config:
      symbols:
        - "output"
      variables:
        ML:LOCAL:TEST_S:
          formula: "output"
```

Then to launch the model we can use the following command:

``` bash
pl --publish -c examples/base/local/deployment_config.yaml`
```        




# Stage 2b - Write a poly-lithic compatible model  - from mlflow

WIP